# UR5 Manipulator Sensor Data — Notebook 03: Feature Engineering

Objective: This notebook transforms the clean, raw sensor data (X) into a rich, predictive feature set by applying domain knowledge, time-series mechanics, and statistical aggregation. The resulting features will maximize the signal available to machine learning models for anomaly detection and prediction.

Input Data:

- Cleaned Sensor Data: **`../data/cleaned/structured_sensor_data.parquet`** (≈153k rows, 73 float64 columns).

Output:

- Feature Set: A significantly wider DataFrame (e.g., ≈250 columns) ready for EDA and modeling.

- Saved to **`../data/features/feature_set.parquet`**.

---


## Step 1: Setup and Data Loading

We begin by loading the necessary libraries and importing the fully cleaned dataset saved from the previous notebook.

In [1]:
import pandas as pd
import numpy as np
import os

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# Define paths
cleaned_data_path = "../data/cleaned/structured_sensor_data.parquet"
output_dir = "../data/features"
os.makedirs(output_dir, exist_ok=True)

In [2]:
# Load the cleaned dataset
try:
    sensor_data = pd.read_parquet(cleaned_data_path)
    print(f"✔ Clean data loaded successfully. Shape: {sensor_data.shape}")
except FileNotFoundError:
    print(f"Error: File not found at {cleaned_data_path}. Please run Notebook 02 first.")

display(sensor_data.head(2))

✔ Clean data loaded successfully. Shape: (153658, 73)


,ROBOT_TIME,ROBOT_TARGET_JOINT_POSITIONS (J1),ROBOT_TARGET_JOINT_POSITIONS (J2),ROBOT_TARGET_JOINT_POSITIONS (J3),ROBOT_TARGET_JOINT_POSITIONS (J4),ROBOT_TARGET_JOINT_POSITIONS (J5),ROBOT_TARGET_JOINT_POSITIONS (J6),ROBOT_ACTUAL_JOINT_POSITIONS (J1),ROBOT_ACTUAL_JOINT_POSITIONS (J2),ROBOT_ACTUAL_JOINT_POSITIONS (J3),ROBOT_ACTUAL_JOINT_POSITIONS (J4),ROBOT_ACTUAL_JOINT_POSITIONS (J5),ROBOT_ACTUAL_JOINT_POSITIONS (J6),ROBOT_TARGET_JOINT_VELOCITIES (J1),ROBOT_TARGET_JOINT_VELOCITIES (J2),ROBOT_TARGET_JOINT_VELOCITIES (J3),ROBOT_TARGET_JOINT_VELOCITIES (J4),ROBOT_TARGET_JOINT_VELOCITIES (J5),ROBOT_TARGET_JOINT_VELOCITIES (J6),ROBOT_ACTUAL_JOINT_VELOCITIES (J1),ROBOT_ACTUAL_JOINT_VELOCITIES (J2),ROBOT_ACTUAL_JOINT_VELOCITIES (J3),ROBOT_ACTUAL_JOINT_VELOCITIES (J4),ROBOT_ACTUAL_JOINT_VELOCITIES (J5),ROBOT_ACTUAL_JOINT_VELOCITIES (J6),ROBOT_TARGET_JOITN_CURRENT (J1),ROBOT_TARGET_JOITN_CURRENT (J2),ROBOT_TARGET_JOITN_CURRENT (J3),ROBOT_TARGET_JOITN_CURRENT (J4),ROBOT_TARGET_JOITN_CURRENT (J5),ROBOT_TARGET_JOITN_CURRENT (J6),ROBOT_ACTUAL_JOINT_CURRENT (J1),ROBOT_ACTUAL_JOINT_CURRENT (J2),ROBOT_ACTUAL_JOINT_CURRENT (J3),ROBOT_ACTUAL_JOINT_CURRENT (J4),ROBOT_ACTUAL_JOINT_CURRENT (J5),ROBOT_ACTUAL_JOINT_CURRENT (J6),ROBOT_TARGET_JOINT_ACCELERATIONS (J1),ROBOT_TARGET_JOINT_ACCELERATIONS (J2),ROBOT_TARGET_JOINT_ACCELERATIONS (J3),ROBOT_TARGET_JOINT_ACCELERATIONS (J4),ROBOT_TARGET_JOINT_ACCELERATIONS (J5),ROBOT_TARGET_JOINT_ACCELERATIONS (J6),ROBOT_TARGET_JOINT_TORQUES (J1),ROBOT_TARGET_JOINT_TORQUES (J2),ROBOT_TARGET_JOINT_TORQUES (J3),ROBOT_TARGET_JOINT_TORQUES (J4),ROBOT_TARGET_JOINT_TORQUES (J5),ROBOT_TARGET_JOINT_TORQUES (J6),ROBOT_JOINT_CONTROL_CURRENT (J1),ROBOT_JOINT_CONTROL_CURRENT (J2),ROBOT_JOINT_CONTROL_CURRENT (J3),ROBOT_JOINT_CONTROL_CURRENT (J4),ROBOT_JOINT_CONTROL_CURRENT (J5),ROBOT_JOINT_CONTROL_CURRENT (J6),ROBOT_CARTESIAN_COORD_TOOL (x),ROBOT_CARTESIAN_COORD_TOOL (y),ROBOT_CARTESIAN_COORD_TOOL (z),ROBOT_CARTESIAN_COORD_TOOL (rx),ROBOT_CARTESIAN_COORD_TOOL (ry),ROBOT_CARTESIAN_COORD_TOOL (rz),ROBOT_TCP_FORCE (x),ROBOT_TCP_FORCE (y),ROBOT_TCP_FORCE (z),ROBOT_TCP_FORCE (rx),ROBOT_TCP_FORCE (ry),ROBOT_TCP_FORCE (rz),ROBOT_JOINT_TEMP (J1),ROBOT_JOINT_TEMP (J2),ROBOT_JOINT_TEMP (J3),ROBOT_JOINT_TEMP (J4),ROBOT_JOINT_TEMP (J5),ROBOT_JOINT_TEMP (J6)
0,747.248,-26.880069,-79.911609,57.095392,-157.771764,-105.009613,-44.724779,-26.87662,-79.910908,57.096775,-157.773152,-105.007564,-44.725462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.210037,-2.213814,-1.589348,-0.162991,0.000451,0.0,0.239874,-3.434454,-1.869670,-0.309583,-0.208931,-0.106753,0.0,0.0,0.0,0.0,0.0,0.0,-1.360105,-25.674171,-18.441131,-1.376068,0.003848,0.0,0.228665,-3.434454,-1.862945,-0.309583,-0.19368,-0.117428,-0.637719,0.277536,0.756995,-1.075034,-1.130315,0.045502,-25.231405,17.439707,6.516588,-1.005161,0.393243,0.969444,25.209991,26.714735,26.71241,29.805393,28.690552,29.992847
1,747.256,-26.880069,-79.911609,57.095392,-157.771764,-105.009613,-44.724779,-26.87662,-79.910225,57.096092,-157.773835,-105.008930,-44.724096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.210037,-2.213814,-1.589348,-0.162991,0.000451,0.0,0.239874,-3.436696,-1.836043,-0.323309,-0.178430,-0.105228,0.0,0.0,0.0,0.0,0.0,0.0,-1.360105,-25.674171,-18.441131,-1.376068,0.003848,0.0,0.237632,-3.434454,-1.840526,-0.309583,-0.19368,-0.117428,-0.637718,0.277543,0.756998,-1.075055,-1.130280,0.045504,-27.052613,17.178792,8.714170,-1.378681,-0.033574,0.297412,25.209991,26.714735,26.71241,29.805393,28.690552,29.992847


## Step 2: Domain-Specific Feature Engineering: Error Metrics

A key indicator of manipulator health is the instantaneous deviation between where the robot is intended to be (Target) and where it actually is (Actual). We create new features for the absolute difference (error) for positions, velocities, and currents across all six joints.

This metric directly captures control system performance and mechanical stress.

In [3]:
# List of sensor types that have both TARGET and ACTUAL values
sensor_types = [
    'JOINT_POSITIONS',
    'JOINT_VELOCITIES',
    # We must include the component that uses the actual column name spelling:
    'JOINT_CURRENT' 
]

# List of joints
joints = ['(J1)', '(J2)', '(J3)', '(J4)', '(J5)', '(J6)']

for s_type in sensor_types:
    for j in joints:
        
        # 1. Determine the correct prefix for the current column based on its known misspelling:
        if s_type == 'JOINT_CURRENT':
            target_prefix = 'ROBOT_TARGET_JOITN_CURRENT' # Uses the JOITN spelling
            actual_prefix = 'ROBOT_ACTUAL_JOINT_CURRENT' # Uses the JOINT spelling
        else:
            target_prefix = f'ROBOT_TARGET_{s_type}'
            actual_prefix = f'ROBOT_ACTUAL_{s_type}'

        # 2. Construct the full column names for Target and Actual
        target_col = f"{target_prefix} {j}"
        actual_col = f"{actual_prefix} {j}"
        error_col = f"ERROR_{s_type}_{j}".replace(' ', '_')
        
        # 3. Calculate the absolute difference
        if target_col in sensor_data.columns and actual_col in sensor_data.columns:
            sensor_data[error_col] = np.abs(sensor_data[target_col] - sensor_data[actual_col])

print(f"✔ Created {len(sensor_types) * len(joints)} new error features.")
print(f"New DataFrame Shape: {sensor_data.shape}")

✔ Created 18 new error features.
New DataFrame Shape: (153658, 91)


## Step 3: Time-Series Feature Engineering: Lagging

Lag features incorporate the robot's past state into the current timestamp, which is essential for predicting the next state or detecting abnormal transitions. We select key performance indicators (KPIs) like position, current, and torque for lagging at short intervals.

In [4]:
# Diagnostic: Print all columns to find the exact name
print([col for col in sensor_data.columns if 'CURRENT' in col])

['ROBOT_TARGET_JOITN_CURRENT (J1)', 'ROBOT_TARGET_JOITN_CURRENT (J2)', 'ROBOT_TARGET_JOITN_CURRENT (J3)', 'ROBOT_TARGET_JOITN_CURRENT (J4)', 'ROBOT_TARGET_JOITN_CURRENT (J5)', 'ROBOT_TARGET_JOITN_CURRENT (J6)', 'ROBOT_ACTUAL_JOINT_CURRENT (J1)', 'ROBOT_ACTUAL_JOINT_CURRENT (J2)', 'ROBOT_ACTUAL_JOINT_CURRENT (J3)', 'ROBOT_ACTUAL_JOINT_CURRENT (J4)', 'ROBOT_ACTUAL_JOINT_CURRENT (J5)', 'ROBOT_ACTUAL_JOINT_CURRENT (J6)', 'ROBOT_JOINT_CONTROL_CURRENT (J1)', 'ROBOT_JOINT_CONTROL_CURRENT (J2)', 'ROBOT_JOINT_CONTROL_CURRENT (J3)', 'ROBOT_JOINT_CONTROL_CURRENT (J4)', 'ROBOT_JOINT_CONTROL_CURRENT (J5)', 'ROBOT_JOINT_CONTROL_CURRENT (J6)', 'ERROR_JOINT_CURRENT_(J1)', 'ERROR_JOINT_CURRENT_(J2)', 'ERROR_JOINT_CURRENT_(J3)', 'ERROR_JOINT_CURRENT_(J4)', 'ERROR_JOINT_CURRENT_(J5)', 'ERROR_JOINT_CURRENT_(J6)']


In [5]:
# --- 3. Time-Series Feature Engineering: Lagging ---

# Select critical columns for lagging
lag_cols = [
    # Verify these names match your sensor_data.columns.tolist() output exactly
    'ROBOT_ACTUAL_JOINT_POSITIONS (J1)', 
    'ROBOT_ACTUAL_JOINT_POSITIONS (J6)', 
    'ROBOT_ACTUAL_JOINT_CURRENT (J1)', 
    'ROBOT_ACTUAL_JOINT_CURRENT (J6)', 
    'ROBOT_TCP_FORCE (x)',
    'ROBOT_TCP_FORCE (z)'
]

# Define lag windows (time steps)
lags = [1, 3, 5, 10]

for col in lag_cols:
    for lag in lags:
        # Create a clean name for the new feature
        new_col_name = f"{col.replace('(', '').replace(')', '').replace(' ', '_')}_LAG_{lag}"
        
        # Shift the column by 'lag' steps
        sensor_data[new_col_name] = sensor_data[col].shift(lag)

print(f"✔ Created {len(lag_cols) * len(lags)} new lag features.")
print(f"New DataFrame Shape: {sensor_data.shape}")

✔ Created 24 new lag features.
New DataFrame Shape: (153658, 115)


## Step 4: Statistical Feature Engineering: Rolling Windows

Rolling window statistics capture the local mean and volatility of the sensor signals. An increase in the rolling standard deviation (std) often precedes a fault, making these features highly predictive.

We use a 50-timestep window, which corresponds to 5 seconds of robot operation (assuming a 10 Hz sampling rate, typical for UR robots).

In [6]:
# Columns to apply rolling features to (Focus on critical operational metrics)
rolling_cols = [
    'ROBOT_ACTUAL_JOINT_CURRENT (J1)', 
    'ROBOT_JOINT_CONTROL_CURRENT (J1)', 
    'ROBOT_ACTUAL_JOINT_VELOCITIES (J1)',
    'ROBOT_TCP_FORCE (x)', 
    'ROBOT_TCP_FORCE (z)',
    'ERROR_JOINT_POSITIONS_(J1)' # Use one of the new error features
]

window = 50 # 50 timesteps (e.g., 5 seconds of data)

for col in rolling_cols:
    base_name = col.replace('(', '').replace(')', '').replace(' ', '_')
    
    # Calculate Rolling Mean (Trend)
    sensor_data[f"{base_name}_ROLL_MEAN_{window}"] = (
        sensor_data[col].rolling(window=window).mean()
    )
    
    # Calculate Rolling Standard Deviation (Volatility/Noise)
    sensor_data[f"{base_name}_ROLL_STD_{window}"] = (
        sensor_data[col].rolling(window=window).std()
    )

print(f"✔ Created {len(rolling_cols) * 2} new rolling features.")
print(f"Final DataFrame Shape: {sensor_data.shape}")

✔ Created 12 new rolling features.
Final DataFrame Shape: (153658, 127)


## Step 5: Final Inspection, Cleanup, and Null Handling

The feature engineering process (especially lagging and rolling calculations) introduces new null values at the start of the time series. We must handle these boundary NaNs before proceeding. Since these NaNs are concentrated at the beginning, we use forward-fill (ffill) to impute them, followed by backward-fill (bfill) for any remaining leading NaNs

In [7]:
# Identify new NaNs introduced by lagging and rolling
new_nulls = sensor_data.isnull().sum().sum()
print(f"Nulls introduced by feature engineering: {new_nulls}")

# Handle boundary NaNs using FFILL and BFILL
sensor_data.ffill(inplace=True)
sensor_data.bfill(inplace=True)

final_nulls = sensor_data.isnull().sum().sum()
print(f"✔ Final null count after imputation: {final_nulls}")

print("\n--- Final Feature Set Preview ---")
display(sensor_data.head())

Nulls introduced by feature engineering: 702
✔ Final null count after imputation: 0

--- Final Feature Set Preview ---


,ROBOT_TIME,ROBOT_TARGET_JOINT_POSITIONS (J1),ROBOT_TARGET_JOINT_POSITIONS (J2),ROBOT_TARGET_JOINT_POSITIONS (J3),ROBOT_TARGET_JOINT_POSITIONS (J4),ROBOT_TARGET_JOINT_POSITIONS (J5),ROBOT_TARGET_JOINT_POSITIONS (J6),ROBOT_ACTUAL_JOINT_POSITIONS (J1),ROBOT_ACTUAL_JOINT_POSITIONS (J2),ROBOT_ACTUAL_JOINT_POSITIONS (J3),ROBOT_ACTUAL_JOINT_POSITIONS (J4),ROBOT_ACTUAL_JOINT_POSITIONS (J5),ROBOT_ACTUAL_JOINT_POSITIONS (J6),ROBOT_TARGET_JOINT_VELOCITIES (J1),ROBOT_TARGET_JOINT_VELOCITIES (J2),ROBOT_TARGET_JOINT_VELOCITIES (J3),ROBOT_TARGET_JOINT_VELOCITIES (J4),ROBOT_TARGET_JOINT_VELOCITIES (J5),ROBOT_TARGET_JOINT_VELOCITIES (J6),ROBOT_ACTUAL_JOINT_VELOCITIES (J1),ROBOT_ACTUAL_JOINT_VELOCITIES (J2),ROBOT_ACTUAL_JOINT_VELOCITIES (J3),ROBOT_ACTUAL_JOINT_VELOCITIES (J4),ROBOT_ACTUAL_JOINT_VELOCITIES (J5),ROBOT_ACTUAL_JOINT_VELOCITIES (J6),ROBOT_TARGET_JOITN_CURRENT (J1),ROBOT_TARGET_JOITN_CURRENT (J2),ROBOT_TARGET_JOITN_CURRENT (J3),ROBOT_TARGET_JOITN_CURRENT (J4),ROBOT_TARGET_JOITN_CURRENT (J5),ROBOT_TARGET_JOITN_CURRENT (J6),ROBOT_ACTUAL_JOINT_CURRENT (J1),ROBOT_ACTUAL_JOINT_CURRENT (J2),ROBOT_ACTUAL_JOINT_CURRENT (J3),ROBOT_ACTUAL_JOINT_CURRENT (J4),ROBOT_ACTUAL_JOINT_CURRENT (J5),ROBOT_ACTUAL_JOINT_CURRENT (J6),ROBOT_TARGET_JOINT_ACCELERATIONS (J1),ROBOT_TARGET_JOINT_ACCELERATIONS (J2),ROBOT_TARGET_JOINT_ACCELERATIONS (J3),ROBOT_TARGET_JOINT_ACCELERATIONS (J4),ROBOT_TARGET_JOINT_ACCELERATIONS (J5),ROBOT_TARGET_JOINT_ACCELERATIONS (J6),ROBOT_TARGET_JOINT_TORQUES (J1),ROBOT_TARGET_JOINT_TORQUES (J2),ROBOT_TARGET_JOINT_TORQUES (J3),ROBOT_TARGET_JOINT_TORQUES (J4),ROBOT_TARGET_JOINT_TORQUES (J5),ROBOT_TARGET_JOINT_TORQUES (J6),ROBOT_JOINT_CONTROL_CURRENT (J1),ROBOT_JOINT_CONTROL_CURRENT (J2),ROBOT_JOINT_CONTROL_CURRENT (J3),ROBOT_JOINT_CONTROL_CURRENT (J4),ROBOT_JOINT_CONTROL_CURRENT (J5),ROBOT_JOINT_CONTROL_CURRENT (J6),ROBOT_CARTESIAN_COORD_TOOL (x),ROBOT_CARTESIAN_COORD_TOOL (y),ROBOT_CARTESIAN_COORD_TOOL (z),ROBOT_CARTESIAN_COORD_TOOL (rx),ROBOT_CARTESIAN_COORD_TOOL (ry),ROBOT_CARTESIAN_COORD_TOOL (rz),ROBOT_TCP_FORCE (x),ROBOT_TCP_FORCE (y),ROBOT_TCP_FORCE (z),ROBOT_TCP_FORCE (rx),ROBOT_TCP_FORCE (ry),ROBOT_TCP_FORCE (rz),ROBOT_JOINT_TEMP (J1),ROBOT_JOINT_TEMP (J2),ROBOT_JOINT_TEMP (J3),ROBOT_JOINT_TEMP (J4),ROBOT_JOINT_TEMP (J5),ROBOT_JOINT_TEMP (J6),ERROR_JOINT_POSITIONS_(J1),ERROR_JOINT_POSITIONS_(J2),ERROR_JOINT_POSITIONS_(J3),ERROR_JOINT_POSITIONS_(J4),ERROR_JOINT_POSITIONS_(J5),ERROR_JOINT_POSITIONS_(J6),ERROR_JOINT_VELOCITIES_(J1),ERROR_JOINT_VELOCITIES_(J2),ERROR_JOINT_VELOCITIES_(J3),ERROR_JOINT_VELOCITIES_(J4),ERROR_JOINT_VELOCITIES_(J5),ERROR_JOINT_VELOCITIES_(J6),ERROR_JOINT_CURRENT_(J1),ERROR_JOINT_CURRENT_(J2),ERROR_JOINT_CURRENT_(J3),ERROR_JOINT_CURRENT_(J4),ERROR_JOINT_CURRENT_(J5),ERROR_JOINT_CURRENT_(J6),ROBOT_ACTUAL_JOINT_POSITIONS_J1_LAG_1,ROBOT_ACTUAL_JOINT_POSITIONS_J1_LAG_3,ROBOT_ACTUAL_JOINT_POSITIONS_J1_LAG_5,ROBOT_ACTUAL_JOINT_POSITIONS_J1_LAG_10,ROBOT_ACTUAL_JOINT_POSITIONS_J6_LAG_1,ROBOT_ACTUAL_JOINT_POSITIONS_J6_LAG_3,ROBOT_ACTUAL_JOINT_POSITIONS_J6_LAG_5,ROBOT_ACTUAL_JOINT_POSITIONS_J6_LAG_10,ROBOT_ACTUAL_JOINT_CURRENT_J1_LAG_1,ROBOT_ACTUAL_JOINT_CURRENT_J1_LAG_3,ROBOT_ACTUAL_JOINT_CURRENT_J1_LAG_5,ROBOT_ACTUAL_JOINT_CURRENT_J1_LAG_10,ROBOT_ACTUAL_JOINT_CURRENT_J6_LAG_1,ROBOT_ACTUAL_JOINT_CURRENT_J6_LAG_3,ROBOT_ACTUAL_JOINT_CURRENT_J6_LAG_5,ROBOT_ACTUAL_JOINT_CURRENT_J6_LAG_10,ROBOT_TCP_FORCE_x_LAG_1,ROBOT_TCP_FORCE_x_LAG_3,ROBOT_TCP_FORCE_x_LAG_5,ROBOT_TCP_FORCE_x_LAG_10,ROBOT_TCP_FORCE_z_LAG_1,ROBOT_TCP_FORCE_z_LAG_3,ROBOT_TCP_FORCE_z_LAG_5,ROBOT_TCP_FORCE_z_LAG_10,ROBOT_ACTUAL_JOINT_CURRENT_J1_ROLL_MEAN_50,ROBOT_ACTUAL_JOINT_CURRENT_J1_ROLL_STD_50,ROBOT_JOINT_CONTROL_CURRENT_J1_ROLL_MEAN_50,ROBOT_JOINT_CONTROL_CURRENT_J1_ROLL_STD_50,ROBOT_ACTUAL_JOINT_VELOCITIES_J1_ROLL_MEAN_50,ROBOT_ACTUAL_JOINT_VELOCITIES_J1_ROLL_STD_50,ROBOT_TCP_FORCE_x_ROLL_MEAN_50,ROBOT_TCP_FORCE_x_ROLL_STD_50,ROBOT_TCP_FORCE_z_ROLL_MEAN_50,ROBOT_TCP_FORCE_z_ROLL_STD_50,ERROR_JOINT_POSITIONS_J1_ROLL_MEAN_50,ERROR_JOINT_POSITIONS

## Step 6: Save Final Feature Set

The data is now feature-rich and ready for the intensive analytical steps in the next notebook. We save the final DataFrame in the highly efficient Parquet format.

In [8]:
# Define final output path
feature_set_path = os.path.join(output_dir, "feature_set.parquet")

# Save the final feature set
sensor_data.to_parquet(feature_set_path, index=False)

print(f"✔ Final feature set saved successfully. Total columns: {sensor_data.shape[1]}")
print(f"Saved to: {feature_set_path}")

✔ Final feature set saved successfully. Total columns: 127
Saved to: ../data/features/feature_set.parquet


# Summary

- Domain Features: Calculated error metrics (Target vs. Actual) for position, velocity, and current across all joints.

- Time-Series Features: Created lag features (1, 3, 5, 10 timesteps) for critical KPIs.

- Statistical Features: Created rolling mean and standard deviation features over a 50-timestep window to capture trend and volatility.

- Output: The final feature-rich dataset is saved, ready for in-depth EDA and modeling in the next notebooks.

---

Next Notebook → `04_deep_exploratory_data_analysis.ipynb`